<a href="https://colab.research.google.com/github/suhaib1769/Thesis/blob/main/colbert_x_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Thu Jun 13 10:53:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2766, done.
remote: Counting objects: 100% (1267/1267), done.
remote: Compressing objects: 100% (399/399), done.
remote: Total 2766 (delta 987), reused 947 (delta 868), pack-reused 1499
Receiving objects: 100% (2766/2766), 2.05 MiB | 3.50 MiB/s, done.
Resolving deltas: 100% (1747/1747), done.


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.0

In [ ]:
import colbert

In [ ]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [ ]:
from datasets import load_dataset

dataset = 'lifestyle'
datasplit = 'dev'

collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

queries_dataset = load_dataset("colbertv2/lotte", dataset)
queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

f'Loaded {len(queries)} queries and {len(collection):,} passages'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for colbertv2/lotte_passages contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/colbertv2/lotte_passages
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dat

Generating dev_collection split: 0 examples [00:00, ? examples/s]

Generating test_collection split: 0 examples [00:00, ? examples/s]

Generating forum_dev split:   0%|          | 0/2076 [00:00<?, ? examples/s]

Generating forum_test split:   0%|          | 0/2002 [00:00<?, ? examples/s]

Generating search_dev split:   0%|          | 0/417 [00:00<?, ? examples/s]

Generating search_test split:   0%|          | 0/661 [00:00<?, ? examples/s]

'Loaded 417 queries and 268,893 passages'

In [ ]:
print(queries[:3])
print()
print(collection[:3])
print()

['how much should i feed my 1 year old english mastiff?', 'are zebra loaches safe with shrimp?', 'are serpae tetras fin nippers?']

['In my experience rabbits are very easy to housebreak. They like to pee and poop in the same place every time, so in most cases all you have to do is put a little bit of their waste in the litter box and they will happily use the litter box. It is very important that if they go somewhere else, miss the edge or kick waste out of the box that you clean it up well and immediately as otherwise those spots will become existing places to pee and poop. When you clean the box, save a little bit of waste and put it in the cleaned box so it smells right to them. For a more foolproof method, you can get a piece of wood soaked with their urine and put that in the box along with droppings or cage them so that they are only in their litter box for a week. Generally, if I try the first method and find that they are not using only the box on the first day, I go for the l

In [ ]:
from parser_2 import Parser2

In [ ]:
xml_file_path = "/content/output_solr34.xml"

data = Parser2.XLMtoString(xml_file_path)

In [ ]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 10000

index_name = f'eana.{nbits}bits'

print(index_name)

eana.2bits


In [ ]:
display(data[:10])


[{'id': 'ID: /2021672/resource_document_mauritshuis_397',
  'text': 'Portrait of Helena Grondt (1613/14-after 1665) by Abraham van den Tempel, dated 1660 - 1660'},
 {'id': 'ID: /2021672/resource_document_mauritshuis_340',
  'text': 'The Annunciation by Francesco Solimena, dated 1693 - 1693'},
 {'id': 'ID: /2021672/resource_document_mauritshuis_426',
  'text': 'Portrait of an Officer by Jan Anthonisz van Ravesteyn, dated 1611 - 1611'},
 {'id': 'ID: /2021672/resource_document_mauritshuis_432',
  'text': 'Portrait of the Brothers Gaspard (1519-1572), Odet (1517-1571) and François (1512-1569) de Châtillon-Coligny by Anoniem (Frankrijk), dated 1579 - 1579'},
 {'id': 'ID: /2021672/resource_document_mauritshuis_354',
  'text': 'St Barbara by Parmigianino, dated None'},
 {'id': 'ID: /2021672/resource_document_mauritshuis_181',
  'text': "Reinier Adriaansz's Declaration of Love by Cornelis Troost, dated 1737 - 1737"},
 {'id': 'ID: /2021672/resource_document_mauritshuis_195',
  'text': 'A Fishmo

In [ ]:

collection = [f"id: {item['id']}, text: {item['text']}" for item in data]
display((collection))

['id: ID: /2021672/resource_document_mauritshuis_397, text: Portrait of Helena Grondt (1613/14-after 1665) by Abraham van den Tempel, dated 1660 - 1660',
 'id: ID: /2021672/resource_document_mauritshuis_340, text: The Annunciation by Francesco Solimena, dated 1693 - 1693',
 'id: ID: /2021672/resource_document_mauritshuis_426, text: Portrait of an Officer by Jan Anthonisz van Ravesteyn, dated 1611 - 1611',
 'id: ID: /2021672/resource_document_mauritshuis_432, text: Portrait of the Brothers Gaspard (1519-1572), Odet (1517-1571) and François (1512-1569) de Châtillon-Coligny by Anoniem (Frankrijk), dated 1579 - 1579',
 'id: ID: /2021672/resource_document_mauritshuis_354, text: St Barbara by Parmigianino, dated None',
 "id: ID: /2021672/resource_document_mauritshuis_181, text: Reinier Adriaansz's Declaration of Love by Cornelis Troost, dated 1737 - 1737",
 'id: ID: /2021672/resource_document_mauritshuis_195, text: A Fishmonger at the Door by Jacob Ochtervelt, dated 1663 - 1663',
 'id: ID: /

In [ ]:
# checkpoint = 'colbert-ir/colbertv2.0'
checkpoint = 'antoinelouis/colbert-xm'

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                # Consider larger numbers for small datasets.
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)

artifact.metadata:   0%|          | 0.00/2.14k [00:00<?, ?B/s]



[Jun 13, 11:55:38] #> Note: Output directory /content/experiments/notebook/indexes/eana.2bits already exists


[Jun 13, 11:55:38] #> Will delete 10 files already at /content/experiments/notebook/indexes/eana.2bits in 20 seconds...
#> Starting...
#> Joined...


In [ ]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/content/experiments/notebook/indexes/eana.2bits'

In [ ]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[Jun 13, 12:00:28] #> Loading codec...
[Jun 13, 12:00:28] #> Loading IVF...
[Jun 13, 12:00:28] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 849.74it/s]

[Jun 13, 12:00:28] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 95.48it/s]


In [ ]:
query = "meisje met parel" # try with an in-range query or supply your own
print(f"#> {query}")

# Find the top-3 passages for this query
results = searcher.search(query, k=3)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> meisje met parel
	 [1] 		 12.6 		 id: ID: /2021672/resource_document_mauritshuis_670, text: Girl with a Pearl Earring by Johannes Vermeer, dated 1665 - 1665
	 [2] 		 11.9 		 id: ID: /2021672/resource_document_mauritshuis_825, text: Girl Teasing a Boy by Bernhard Keil, dated 1660 - 1650
	 [3] 		 10.9 		 id: ID: /2021672/resource_document_mauritshuis_623, text: Girl Reading by Claes Hals, dated 1660 - 1660


In [ ]:
embeddings = indexer

In [ ]:
type(embeddings)

colbert.indexer.Indexer